<a href="https://colab.research.google.com/github/SukruOnurYLMZ/CNN-Onur/blob/main/HELMET_DETECTION_WITH_YOLOv12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**🎓 Computer Vision – YOLOv12 Helmet Object Detector Final Assignment ONUR**

This notebook presents the results of my project titled “Helmet Detection”, an object detection model developed and trained using the Roboflow platform. The main objective of this project is to detect face masks in real-world images, reflecting scenarios experienced during the pandemic.

The model was trained using a custom-labeled dataset consisting of over 380 images, annotated with bounding boxes for "mask" and "no-mask" categories.

Model Type: Roboflow 3.0 Object Detection (Fast)

You can preview the model and its deployment features through Roboflow’s hosted interface.

This project is open for collaboration, and I would be excited to work with others who are interested in:

Enhancing the dataset or model architecture (e.g., YOLOv12, MobileNet, etc.)

Deploying the model for real-time or embedded system use

Conducting joint research or building AI applications focused on public health and safety

Integrating the system with video streams or surveillance tools

If you're interested in contributing or partnering, feel free to connect with me.

Project Owner: Onur Yilmaz Platform: Roboflow Model Name: Helmet Detection 1 Date: July 9, 2025**

🧠 Project Tasks:

In [ ]:
!pip install -q git+https://github.com/sunsmarterjie/yolov12.git roboflow supervision flash-attn

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="ZA7VGxpX0FGbFDZHJAnp")
project = rf.workspace("onur-yilmaz-3unus").project("helmet-esqea-yk2q6")
version = project.version(1)
dataset = version.download("yolov12")

In [ ]:
!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!echo -e "test: {dataset.location}/test/images\ntrain: {dataset.location}/train/images\nval: {dataset.location}/valid/images" >> {dataset.location}/data.yaml

**2. Train a YOLOv12 Model:**

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov12m.pt')  # You can use yolov12n.pt, yolov12m.pt etc.
model.train(data=f"{dataset.location}/data.yaml", epochs=18)

**3. Evaluate the Model:**

In [ ]:
import supervision as sv
from supervision.metrics import MeanAveragePrecision
import matplotlib.pyplot as plt
import numpy as np

ds = sv.DetectionDataset.from_yolo(
    images_directory_path=f"{dataset.location}/test/images",
    annotations_directory_path=f"{dataset.location}/test/labels",
    data_yaml_path=f"{dataset.location}/data.yaml"
)

model = YOLO(f"/content/runs/detect/train/weights/best.pt")

predictions = []
targets = []

for _, image, target in ds:
    results = model(image, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    predictions.append(detections)
    targets.append(target)

map = MeanAveragePrecision().update(predictions, targets).compute()
print("mAP 50:95", map.map50_95)
print("mAP 50", map.map50)
print("mAP 75", map.map75)

map.plot()


Overall Metrics (Top Left):

mAP@50:95 (average mAP over IoU thresholds from 0.5 to 0.95) is 0.426.

mAP@50 (mAP at IoU threshold 0.5) is 0.831.

mAP@75 (mAP at IoU threshold 0.75) is 0.364.

Breakdown by Object Size and Metric (Bar Plot): The plot categorizes performance by object size: Small, Medium, and Large, and for each, it shows three mAP metrics: mAP@50-95, mAP@50, and mAP@75.

Small Objects:

All mAP values (mAP@50-95, mAP@50, mAP@75) for small objects are 0.00, indicating no successful detections or very poor performance for this size category.

Medium Objects:

mAP@50-95 for medium objects is 0.47.

mAP@50 for medium objects is 0.86, which is the highest individual mAP@50 value across all categories.

mAP@75 for medium objects is 0.51.

Large Objects:

mAP@50-95 for large objects is 0.40.

mAP@50 for large objects is 0.77.

mAP@75 for large objects is 0.34.

**4. Test the Model on an Unseen Image:**

In [ ]:
import random

i = random.randint(0, len(ds))
image_path, image, target = ds[i]

results = model(image, verbose=False)[0]
detections = sv.Detections.from_ultralytics(results).with_nms()

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

annotated_image = image.copy()
annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

sv.plot_image(annotated_image)

In [ ]:
import random

i = random.randint(0, len(ds))
image_path, image, target = ds[i]

results = model(image, verbose=False)[0]
detections = sv.Detections.from_ultralytics(results).with_nms()

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

annotated_image = image.copy()
annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

sv.plot_image(annotated_image)

In [ ]:
import matplotlib.pyplot as plt

# Academic-quality font and style
plt.rcParams.update({
    "font.size": 12,
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "figure.figsize": (7, 5),
    "font.family": "serif"
})

# Prepare data
scores = {
    "mAP@50:95": map.map50_95,
    "mAP@50": map.map50,
    "mAP@75": map.map75
}

# Plot
fig, ax = plt.subplots()
bars = ax.bar(scores.keys(), scores.values(), color=["#4c72b0", "#55a868", "#c44e52"])

# Add value labels on top
for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{height:.2f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 5),  # vertical offset
                textcoords="offset points",
                ha='center', va='bottom')

# Titles and labels
ax.set_title("Object Detection Performance Metrics", pad=15)
ax.set_ylabel("Score")
ax.set_ylim(0, 1.05)
ax.grid(axis='y', linestyle='--', linewidth=0.5, alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import os
import glob

# Load YOLOv12 model
model = YOLO("/content/runs/detect/train/weights/best.pt")

# Dynamically find an image in the test set
test_images_dir = f"{dataset.location}/test/images"
image_files = glob.glob(os.path.join(test_images_dir, '*.*')) # Find all files in the directory

if not image_files:
    print(f"Error: No image files found in {test_images_dir}")
else:
    # Use the first image found as an example
    image_path = image_files[0]
    print(f"Using image: {image_path}")

    # Run inference
    results = model(image_path)

    # Show result
    res_plotted = results[0].plot()
    plt.imshow(res_plotted[..., ::-1])
    plt.axis('off')
    plt.title("Detection Result")
    plt.show()

**5. Test the Model on a Short Video:**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Upload a video
from google.colab import files
uploaded_video = files.upload()

# Get video filename
video_path = next(iter(uploaded_video))

# Run inference
model.predict(source=video_path, save=True, save_txt=True, conf=0.25)

# Output will be saved under /content/runs/detect/predict/

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os

# Your video path from the previous output
video_path = '/content/1721568-hd_1920_1080_30fps.mp4'

# --- Choose which single frame you want ---
# Frame numbers start from 0. Your video has 263 frames (0 to 262).
# Let's say you want to 'post' frame number 150.
frame_to_extract = 112 # You can change this to any number between 0 and 262

# --- Extract and display/save the frame ---
if not os.path.exists(video_path):
    print(f"Error: Video file not found at '{video_path}'")
else:
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
    else:
        # Set the video to the desired frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_to_extract)

        # Read the frame
        ret, frame = cap.read()

        if ret:
            print(f"Successfully extracted frame {frame_to_extract}")

            # 1. Display the frame in your Colab output
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(10, 8))
            plt.imshow(frame_rgb)
            plt.title(f"Single Frame: {frame_to_extract}")
            plt.axis('off') # Hide axes
            plt.show()

            # 2. Save the frame as an image file (e.g., JPEG)
            output_image_filename = f'extracted_frame_{frame_to_extract}.jpg'
            cv2.imwrite(output_image_filename, frame) # Saves in BGR format
            print(f"Frame also saved as '{output_image_filename}' in your Colab environment.")

        else:
            print(f"Error: Could not read frame {frame_to_extract} from video. It might be out of range.")

        cap.release()
        print("Video capture released.")

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import glob

# --- Step 1: Find the directory where results were saved ---
output_base_dir = 'runs/detect'
predict_dirs = sorted(glob.glob(os.path.join(output_base_dir, 'predict*')))

if not predict_dirs:
    print(f"Error: No prediction directories found in '{output_base_dir}'.")
    print("Please ensure `model.predict(save=True)` was run successfully.")
else:
    latest_predict_dir = predict_dirs[-1] # Get the latest (alphabetically/numerically last)
    print(f"Using results from: {latest_predict_dir}")

    # --- Step 2: Construct the expected path to the output video ---
    # Use the actual filename of the video that was uploaded and processed
    video_filename = '/content/runs/detect/predict/6473954-uhd_2160_3840_25fps.avi' # Corrected video filename
    output_video_with_detections_path = os.path.join(latest_predict_dir, video_filename)

    # --- Choose which single frame you want ---
    # Frame numbers start from 0. The video has 805 frames (0 to 804).
    frame_number_to_display = 400 # <--- Change this number to the frame you want to display

    # --- Step 4: Open the output video and extract the frame ---
    if not os.path.exists(output_video_with_detections_path):
        print(f"Error: Output video with detections not found at '{output_video_with_detections_path}'")
        print("This might happen if the original video name was different or if the output format changed.")
    else:
        print(f"Opening output video: {output_video_with_detections_path}")
        cap = cv2.VideoCapture(output_video_with_detections_path)

        if not cap.isOpened():
            print(f"Error: Could not open the output video file {output_video_with_detections_path}.")
            print("Ensure OpenCV can handle .avi files (might need additional codecs if on local machine, but usually fine in Colab).")
        else:
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            print(f"Total frames in output video: {total_frames}")

            if frame_number_to_display >= total_frames or frame_number_to_display < 0:
                print(f"Warning: Frame {frame_number_to_display} is out of bounds. Please choose a frame between 0 and {total_frames - 1}.")
                frame_number_to_display = 0 # Default to first frame
                print(f"Extracting the first frame (0) instead.")


            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number_to_display)


            ret, frame = cap.read()

            if ret:
                print(f"Successfully extracted frame {frame_number_to_display} with detections.")

                # Convert BGR (OpenCV default) to RGB for Matplotlib display
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # 5. Display the frame in your Colab output
                plt.figure(figsize=(15, 10)) # Adjust figure size as needed
                plt.imshow(frame_rgb)
                plt.title(f"Frame {frame_number_to_display} with Detections")
                plt.axis('off') # Hide axes for a cleaner view
                plt.show()

                # 6. Save the frame as an image file (optional)
                output_image_filename = f'detected_frame_{frame_number_to_display}.jpg'
                cv2.imwrite(output_image_filename, frame) # Saves in BGR format
                print(f"Frame also saved as '{output_image_filename}' in your Colab environment.")

            else:
                print(f"Error: Could not read frame {frame_number_to_display} from the output video.")

            cap.release()
            print("Video capture released.")

In [ ]:
# Upload a video
from google.colab import files
uploaded_video = files.upload()

# Get video filename
video_path = next(iter(uploaded_video))

# Run inference
model.predict(source=video_path, save=True, save_txt=True, conf=0.25)

# Output will be saved under /content/runs/detect/predict/

**📝 Project Summary – Pandemic Time Mask Detection**

This project focuses on detecting face masks in images using object detection techniques. The model was trained on a custom dataset with 386 annotated images using Roboflow’s Object Detection (Fast) model.


**--🔍 Model Performance:--**


mAP@50: 87.5%

Precision: 88.6%

Recall: 78.6%

Model Type: Roboflow 3.0 (Fast)

The model performs well in identifying individuals wearing masks across various environments. It's suitable for real-time applications like workplace safety monitoring, smart cameras, or educational purposes.

**🤝 Collaboration:**

This project is open for collaboration in areas such as:

Improving model accuracy with YOLOv12 or other architectures

Expanding the dataset with domain-specific images

Deployment to web/mobile/IoT platforms

Joint research or academic publishing